##### Otimizando função de coeficientes

In [1]:
from functools import reduce
from sage.all import binomial
def dicionario_geral(poly):
    """INPUT - poly- Um polinomio como expressão simbolica na classe <class 'sage.symbolic.expression.Expression'>
    OUTPUT - Um dicionario com CHAVE do tipo tupla multigrau (d1,d2,...,dl) correpondendo ao monomio 
    variavel1^(d1)*variavel2^(d2)*...*variavell^(dl) e VALOR sendo o coeficiente deste monomio.
    OBSERVACAO- A ordem das variaveis imspota pelo SAGE manda na ordem das chaves do dicionario. 
    Exemplo-
    sage: z, y, x, u = var('z y x u')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = p.free_variables(); variaveis
        (u,x,y,z)
    sage: dicionario_geral(p)
        {(0, 2, 0, 2): 202,
         (1, 1, 0, 1): 1101,
         (2, 0, 2, 0): 2020,
         (2, 1, 0, 0): 2100,
         (3, 0, 2, 2): 3022,
         (3, 0, 3, 1): 3031}
        
        
    sage: u,v = var('u v')
    sage: p3 = 23*u^2*v^3+21*u^2*v+11*u*v+10*u+4;
    sage: dicionario_geral(p3)
      {(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}
    """
    def nova_lista(lista, var):
        new_lista = [];
        for item in lista:
            sublista = item[0].coefficients(var)
            for subitem in sublista:
                subitem = [subitem[0],subitem[1:]+item[-1]]
                new_lista.append(subitem)
        return  new_lista

    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
#####################################################################
    variavel_1 = variaveis[0];
    l_variavel_1 = (poly).coefficients(variavel_1);
    l = [[item[0],item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(list(reversed(j[1]))) : j[0] for j in l}
    return dicionario
from functools import reduce

def coeff_de_moebius(poly, graus, graus_total, extremos, dic_coeffs):    
    soma_total = 0

    binomials_cache = {}
    powers_cache = {}

    for chave in novo_coefsxy.keys():
        somas = []
        for k in range(len(chave)):
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]

            somak = 0
            for pp in range(ck + 1):
                binom_ck_pp = binomials_cache.get((ck, pp))
                if binom_ck_pp is None:
                    binom_ck_pp = binomial(ck, pp)
                    binomials_cache[(ck, pp)] = binom_ck_pp

                binom_gtck_gkspp = binomials_cache.get((gt - ck, gks - pp))
                if binom_gtck_gkspp is None:
                    binom_gtck_gkspp = binomial(gt - ck, gks - pp)
                    binomials_cache[(gt - ck, gks - pp)] = binom_gtck_gkspp

                a_k_pp = powers_cache.get((a_k, pp))
                if a_k_pp is None:
                    a_k_pp = pow(a_k, pp)
                    powers_cache[(a_k, pp)] = a_k_pp

                b_k_ck_pp = powers_cache.get((b_k, ck - pp))
                if b_k_ck_pp is None:
                    b_k_ck_pp = pow(b_k,(ck - pp))
                    powers_cache[(b_k, ck - pp)] = b_k_ck_pp

                somak += binom_ck_pp * binom_gtck_gkspp * a_k_pp * b_k_ck_pp

            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total += reduce(lambda a, b: a * b, somas_e_valor)

    return soma_total

In [23]:
def moebius_coefficient3(polynomial, degree_per_variable, total_degree, variable_ranges, coefficient_dict):
    """
    Calculates the Möbius coefficient for a given polynomial expression.

    **Parameters:**
    - `poly`: A polynomial as a symbolic expression (class `<class 'sage.symbolic.expression.Expression'>`).
    - `degrees`: A list of natural numbers representing the multidegree of the monomial for which you want the coefficient.
                 **Important:** The order of degrees should follow the order in which Sage orders the variables.
    - `bounds`: A list of pairs `(a_i, b_i)` that give the interval of the Moebius substitution for the variable `x_i`, 
                i.e., `x_i --> x_i = (a_i*u_i + b_i)/(u_i + 1)` for the variable `u_i`.
    - `dic_coeffs`: A dictionary where the keys are tuples of degrees and the values are the coefficients of the monomials.

    **Returns:**
    - The Möbius coefficient for the given polynomial expression.

    **Examples:**
    ```python
    sage: u, x, v, y, z = var('u x v y z')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: d = convert_polynomial_to_dict(p)
    sage: vars = p.variables()
    sage: total_degs = [p.degree(x) for x in vars]
    sage: ranges = [(3, 5), (7, 11), (13, 17), (19, 23)]
    sage: calculate_moebius_coefficient(p, [0, 0, 1, 0], total_degs, ranges, d)
    244374065874

    sage: u, v = var('u v')
    sage: p = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4
    sage: vars = p.variables()
    sage: total_degs = [p.degree(x) for x in vars]
    sage: degs = [1, 2]  # To find the coefficient of (x_1)^1 * (x_2)^2
    sage: bounds = [(3, 5), (7, 11)]
    sage: d = convert_polynomial_to_dict(p)
    sage: calculate_moebius_coefficient(p, degs, total_degs, bounds, d)
    1133944
    ```
    """
    from sage.all import binomial
    from functools import reduce

    total_sum = 0

    binom_cache = {}
    pow_cache = {}

    for key in coefficient_dict.keys():
        partial_sums = []
        for k in range(len(key)):
            ck = key[k]
            tot_deg_k = total_degree[k]
            deg_k = degree_per_variable[k]
            a_k, b_k = variable_ranges[k]

            partial_sum = 0
            for pp in range(ck + 1):
                binom_ck_pp = binom_cache.get((ck, pp))
                if binom_ck_pp is None:
                    binom_ck_pp = binomial(ck, pp)
                    binom_cache[(ck, pp)] = binom_ck_pp

                binom_tot_deg_diff = binom_cache.get((tot_deg_k - ck, deg_k - pp))
                if binom_tot_deg_diff is None:
                    binom_tot_deg_diff = binomial(tot_deg_k - ck, deg_k - pp)
                    binom_cache[(tot_deg_k - ck, deg_k - pp)] = binom_tot_deg_diff

                a_k_pow = pow_cache.get((a_k, pp))
                if a_k_pow is None:
                    a_k_pow = a_k^pp
                    pow_cache[(a_k, pp)] = a_k_pow

                b_k_pow = pow_cache.get((b_k, ck - pp))
                if b_k_pow is None:
                    b_k_pow = b_k^(ck - pp)
                    pow_cache[(b_k, ck - pp)] = b_k_pow

                partial_sum += binom_ck_pp * binom_tot_deg_diff * a_k_pow * b_k_pow

            partial_sums.append(partial_sum)
        partial_sums_and_value = partial_sums + [coefficient_dict[key]]
        total_sum += reduce(lambda a, b: a * b, partial_sums_and_value)

    return total_sum

In [4]:
load("util.sage")

In [5]:
var('u, x, v, y, z')
poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
dic =polynomial_to_dict(poly)
variaveis = poly.variables()
graus_total = [poly.degree(x) for x in variaveis]
extremos = [(3,5),(7,11),(13,17),(19,23)]

moebius_coefficient(poly, [0, 0, 1, 0] ,graus_total,  extremos, dic)


244374065874

In [7]:
moebius_coefficient(poly, [0, 0, 1, 0] ,graus_total,  extremos, dic)

244374065874

In [9]:
sage: u, v = var('u v')
sage: poly = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4
variaveis = poly.variables()
graus_total = [poly.degree(x) for x in variaveis]
sage: degrees = [1, 2]  # To find the coefficient of (x_1)^1 * (x_2)^2
sage: bounds = [(3, 5), (7, 11)]
sage: dic = polynomial_to_dict(poly)  # Must be a global variable

In [10]:
moebius_coefficient(poly,degrees, graus_total, bounds, dic)

1133944

#### Teste robusto

In [11]:
poly = load("poly_uvt")
n_uplas = load("n_uplas")
dic = load("dic")
extremos = load("extremos")

def calculate_moebius_coefficient2(polynomial, degree_per_variable, total_degree, variable_ranges, coefficient_dict):
    
    from sage.all import binomial
    from functools import reduce

    total_sum = 0

    binom_cache = {}
    pow_cache = {}

    def get_binom(n, k):
        if (n, k) not in binom_cache:
            binom_cache[(n, k)] = binomial(n, k)
        return binom_cache[(n, k)]

    def get_pow(base, exp):
        if (base, exp) not in pow_cache:
            pow_cache[(base, exp)] = base ** exp
        return pow_cache[(base, exp)]

    for key, coef in coefficient_dict.items():
        partial_sums = []
        for k, ck in enumerate(key):
            tot_deg_k = total_degree[k]
            deg_k = degree_per_variable[k]
            a_k, b_k = variable_ranges[k]

            partial_sum = sum(
                get_binom(ck, pp) * get_binom(tot_deg_k - ck, deg_k - pp) * get_pow(a_k, pp) * get_pow(b_k, ck - pp)
                for pp in range(ck + 1)
            )

            partial_sums.append(partial_sum)
        total_sum += coef * reduce(lambda a, b: a * b, partial_sums)

    return total_sum

In [12]:
variaveis = poly.variables()
graus_total = [poly.degree(x) for x in variaveis]
chaves = dic.keys()
novo_coefsxy = dic

In [14]:
%timeit coeff = [moebius_coefficient(poly, grau, graus_total, extremos, novo_coefsxy) for grau in n_uplas]

14.6 s ± 203 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
len(n_uplas)

400